In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
from data.audio_dataset import AudioSpectrogramDataset
from models.GAN import GAN

import torch
import librosa
import soundfile as sf
from torch.utils.data import DataLoader

In [3]:
# # hyperparameters
# z_dim = 100
batch_size = 32
# num_epochs = 100
# learning_rate = 0.0002
# beta1 = 0.5

# base_directory = "../data/raw/NSynth/audio"
base_directory = "../data/raw/Bass"

dataset = AudioSpectrogramDataset(base_directory=base_directory, spectro_type='mel')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
MusicGAN = GAN()

MusicGAN.train(dataloader=dataloader)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x22528 and 45056x1)

In [ ]:
MusicGAN.generator.eval()

In [7]:
z = torch.randn(1, MusicGAN.latent_dim).to(MusicGAN.device)
generated_spectrogram = MusicGAN.generator(z)

In [8]:
spectrogram = generated_spectrogram.cpu().detach().numpy()
spectrogram = spectrogram.squeeze()

In [9]:
audio = librosa.feature.inverse.mel_to_audio(spectrogram)

path = '../models/generated_samples/'
file_name = 'generated_audio.wav'

sf.write(path + file_name, audio, 22050)